# ZenML: Open-source MLOps Framework for reproducible ML pipelines

![Test](../_assets/Logo/zenml.svg)

![Sam](../_assets/sam.png)

In [ ]:
from absl import logging as absl_logging
import warnings
warnings.filterwarnings('ignore')
%load_ext autoreload
%autoreload 2
absl_logging.set_verbosity(-10000)

Let's begin by initializing ZenML in our directory. We are going to use a local stack to begin with, for simplicity and then transition to other stacks. This can be achieved in code by executing the following block.

# Initialize ZenML

In [ ]:
!zenml init

## The ZenML stack

The ZenML stack is a concept that describes the union of Metadata Store, Artifact Store and Orchestrator that will be used for all pipeline runs. When you get started with zenml you start off with a default local stack.

In [ ]:
!zenml stack list

### The Local Stack

You can imagine the local stack to look like this. Within the diagram we show how a generic pipeline interacts with the local stack.

![LocalStack](../_assets/localstack.png "LocalStack")

In [ ]:
!zenml stack set local_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run.py

We will now use the Kubeflow integration to extend the concept of stacks

![All](../_assets/evidently+mlflow+discord+kubeflow.png "All")

In [ ]:
!zenml integration install kubeflow -f

### The Kubeflow Pipeline stack

Now we want to transition to a kubeflow stack that will look a little bit like this. Note that for kubeflow pipelines we also need a registry where the docker images for each step are registered. 

![KubeflowStack](../_assets/localstack-with-kubeflow-orchestrator.png "KubeflowStack")

But we have good news! You barely have to do anything to transition.

In [ ]:
# You register a container registry with zenml
!zenml container-registry register local_registry --type=default --uri=localhost:5000
    
# You register an orchestrator with zenml
!zenml orchestrator register kubeflow_orchestrator --type=kubeflow

# Now it all is combined into the local_kubeflow_stack
!zenml stack register local_kubeflow_stack \
    -m local_metadata_store \
    -a local_artifact_store \
    -o kubeflow_orchestrator \
    -c local_registry

# And we activate the new stack, now all pipelines will be run within this stack
!zenml stack set local_kubeflow_stack

# Check it out, your new stack is registered
!zenml stack list

### Starting up your new kubeflow pipelines stack

All that is left to do is power up your stack. This is just one more line away. The stack up process might take some time for you. In the background k3d will be creating and starting up a cluster of docker containers to host kubeflow pipelines locally. 

In [1]:
!zenml stack up

Provisioning resources for stack 'local_kubeflow_stack'.
Provisioning resources for stack 'local_kubeflow_stack'.
Provisioning local Kubeflow Pipelines deployment...
Creating local K3D cluster 'zenml-kubeflow-c41766aa'.
WARN[0000] No node filter specified                     
INFO[0000] Prep: Network                                
INFO[0000] Created network 'k3d-zenml-kubeflow-c41766aa' 
INFO[0000] Created volume 'k3d-zenml-kubeflow-c41766aa-images' 
INFO[0000] Creating node 'k3d-zenml-kubeflow-registry.localhost' 
INFO[0001] Successfully created registry 'k3d-zenml-kubeflow-registry.localhost' 
INFO[0001] Starting new tools node...                   
INFO[0001] Starting Node 'k3d-zenml-kubeflow-c41766aa-tools' 
INFO[0002] Creating node 'k3d-zenml-kubeflow-c41766aa-server-0' 
INFO[0002] Creating LoadBalancer 'k3d-zenml-kubeflow-c41766aa-serverlb' 
INFO[0002] Using the k3d-tools node to gather environment information 
INFO[0003] Starting cluster 'zenml-kubeflow-c41766aa'   
INFO[0003] 

One or more pods not ready yet, waiting for 30 seconds...
Current pod status:
NAME                                              READY   STATUS              RESTARTS   AGE
cache-deployer-deployment-d95f8b79f-gbqp8         0/1     ContainerCreating   0          62s
cache-server-55897df854-wmcjh                     0/1     ContainerCreating   0          62s
minio-5b65df66c9-fhgvh                            0/1     Pending             0          62s
mysql-f7b9b7dd4-k9grv                             0/1     Pending             0          60s
ml-pipeline-viewer-crd-6d4dc67b48-4xgsx           0/1     ContainerCreating   0          61s
workflow-controller-99b6487-7v8bf                 0/1     ContainerCreating   0          60s
ml-pipeline-visualizationserver-75d8c8cd9-5wff5   0/1     ContainerCreating   0          61s
ml-pipeline-ui-78c69869b8-4m52x                   0/1     ContainerCreating   0          61s
ml-pipeline-persistenceagent-69bdb89cfc-l94b8     0/1     ContainerCreating   0      

One or more pods not ready yet, waiting for 30 seconds...
Current pod status:
NAME                                              READY   STATUS              RESTARTS   AGE
cache-deployer-deployment-d95f8b79f-gbqp8         0/1     ContainerCreating   0          4m9s
cache-server-55897df854-wmcjh                     0/1     ContainerCreating   0          4m9s
ml-pipeline-visualizationserver-75d8c8cd9-5wff5   0/1     ContainerCreating   0          4m8s
metadata-writer-5c84d65485-g96v8                  0/1     ImagePullBackOff    0          4m9s
minio-5b65df66c9-fhgvh                            0/1     ContainerCreating   0          4m9s
mysql-f7b9b7dd4-k9grv                             0/1     ContainerCreating   0          4m7s
ml-pipeline-scheduledworkflow-f45d59698-xkf2v     1/1     Running             0          4m8s
ml-pipeline-viewer-crd-6d4dc67b48-4xgsx           1/1     Running             0          4m8s
ml-pipeline-persistenceagent-69bdb89cfc-l94b8     1/1     Running            

One or more pods not ready yet, waiting for 30 seconds...
Current pod status:
NAME                                              READY   STATUS              RESTARTS   AGE
cache-deployer-deployment-d95f8b79f-gbqp8         0/1     ContainerCreating   0          7m15s
cache-server-55897df854-wmcjh                     0/1     ContainerCreating   0          7m15s
ml-pipeline-visualizationserver-75d8c8cd9-5wff5   0/1     ContainerCreating   0          7m14s
metadata-writer-5c84d65485-g96v8                  0/1     ImagePullBackOff    0          7m15s
minio-5b65df66c9-fhgvh                            0/1     ContainerCreating   0          7m15s
ml-pipeline-scheduledworkflow-f45d59698-xkf2v     1/1     Running             0          7m14s
ml-pipeline-viewer-crd-6d4dc67b48-4xgsx           1/1     Running             0          7m14s
metadata-envoy-deployment-5b587ff9d4-br5gk        1/1     Running             0          7m15s
ml-pipeline-ui-78c69869b8-4m52x                   1/1     Running    

One or more pods not ready yet, waiting for 30 seconds...
Current pod status:
NAME                                              READY   STATUS              RESTARTS   AGE
cache-server-55897df854-wmcjh                     0/1     ContainerCreating   0          10m
ml-pipeline-scheduledworkflow-f45d59698-xkf2v     1/1     Running             0          10m
ml-pipeline-viewer-crd-6d4dc67b48-4xgsx           1/1     Running             0          10m
metadata-envoy-deployment-5b587ff9d4-br5gk        1/1     Running             0          10m
ml-pipeline-ui-78c69869b8-4m52x                   1/1     Running             0          10m
workflow-controller-99b6487-7v8bf                 1/1     Running             0          10m
ml-pipeline-persistenceagent-69bdb89cfc-l94b8     1/1     Running             1          10m
mysql-f7b9b7dd4-k9grv                             1/1     Running             0          10m
metadata-grpc-deployment-6b5685488-8ctzj          1/1     Running             5      

If you scroll down all the way on the previous output you should see a link to your running kubeflow pipelines UI. Most probably this will be at [http://localhost:8080/](http://localhost:8080/).

<div class="alert alert-block alert-info">
    <b>Note:</b> Currently running pipelines defined within a jupyter notebook cell is
    not supported. To get around this you can run the train pipeline within this repo. 
</div>

In [ ]:
!zenml stack set local_kubeflow_stack
# Let's train within kubeflow pipelines - this will deploy the training pipeline on a Schedule
!python run.py